In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pylab import rcParams
from math import floor, ceil
from sklearn.preprocessing import OneHotEncoder

In [17]:
from keras.datasets import cifar10

In [18]:
## preprocess data
(x_img_train,y_label_train),(x_img_test,y_label_test) = cifar10.load_data()

In [19]:
def normalize(x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    return x
x_train = normalize(x_img_train).reshape(50000,32*32*3)
x_test = normalize(x_img_test).reshape(10000,32*32*3)

In [20]:
## One hot encoding of labels

def load_label_names():
    return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

encode=OneHotEncoder(dtype=np.float32,sparse=False)

y_train=encode.fit_transform(y_label_train)
y_test=encode.fit_transform(y_label_test)

y_train=np.array(y_train)
y_test=np.array(y_test)

C:\Users\nhors\.conda\envs\tf_cpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\nhors\.conda\envs\tf_cpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto

In [21]:
## Settings

sns.set(style='ticks', palette='Spectral', font_scale=1.5)

material_palette = ["#4CAF50", "#2196F3", "#9E9E9E", "#FF9800", "#607D8B", "#9C27B0"]
sns.set_palette(material_palette)
rcParams['figure.figsize'] = 16, 8

plt.xkcd();
random_state = 42
np.random.seed(random_state)
tf.set_random_seed(random_state)

In [53]:
## wrapper function

def multilayer_perceptron(x, weights, biases, keep_prob):
    
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

## NN shape

n_hidden = 1024
n_input = x_train.shape[1]
n_classes = y_train.shape[1]

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden])),
    'h2': tf.Variable(tf.random_normal([n_hidden, n_hidden])),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden])),
    'b2': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

keep_prob = tf.placeholder("float")

## Hyperparams

training_epochs = 25
display_step = 1
batch_size = 32

x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

In [54]:
## Defining loss function
predictions = multilayer_perceptron(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [46]:
## Resampling indices
x_indices = np.random.choice(50000,128000)
type(x_train)
total_batch = np.concatenate((np.linspace(32,32*100,100),np.linspace(3264,3264+64*149,150),np.linspace(12800+128,12800+128+128*199,200)))
total_batch = np.concatenate((total_batch,np.linspace(38400+256,38400+256+256*249,250),np.linspace(102400+512,102400+512+512*49,50)))
total_batch = np.concatenate(([0],total_batch))

In [54]:
## smush x_train and y_train, random sample x128000, then split

array([19294, 49790, 10774, ..., 17842, 25691, 31388])

In [55]:
## backpropagation of NN
import time
t1=time.time()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        x_batches = np.split(x_train_1, total_batch)
        y_batches = np.split(y_train_1, total_batch)
        for i in range(total_batch):
            batch_y = y_batches[i]
            batch_x = x_batches[i]
            est, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y,
                                keep_prob: 0.8
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    #print ("Validation Accuracy:", accuracy.eval({x: x_test, y: y_test}))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))
    
t2=time.time()
print((t2-t1)/60," minutes")

Epoch: 0001 cost= 2331.063377546
Epoch: 0002 cost= 970.197704828
Epoch: 0003 cost= 724.890203388
Epoch: 0004 cost= 616.160504150
Epoch: 0005 cost= 558.395989487
Epoch: 0006 cost= 535.228945703
Epoch: 0007 cost= 513.560534290
Epoch: 0008 cost= 495.105505874
Epoch: 0009 cost= 476.536110160
Epoch: 0010 cost= 458.960800014
Epoch: 0011 cost= 443.534137844
Epoch: 0012 cost= 428.707146084
Epoch: 0013 cost= 414.534580191
Epoch: 0014 cost= 399.612507928
Epoch: 0015 cost= 386.153957721
Epoch: 0016 cost= 372.469228843
Epoch: 0017 cost= 359.563503619
Epoch: 0018 cost= 347.105698104
Epoch: 0019 cost= 333.920939023
Epoch: 0020 cost= 322.826967849
Epoch: 0021 cost= 312.093698403
Epoch: 0022 cost= 303.386896979
Epoch: 0023 cost= 294.557034837
Epoch: 0024 cost= 283.372540032
Epoch: 0025 cost= 275.534135642
Optimization Finished!
Accuracy: 0.3583
4.506707155704499  minutes
